# MongoDB笔记

### mongodb

pymongo是线程安全的,但是不是fork-safe. 当多进程使用同一个client实例会报错:

解决办法:
在连接mongo数据库时,设置关键字connect=False,即在mongodb实例化时不建立连接,等到有数据库操作才进行数据库连接.

In [ ]:
class MongoBaseModel(object):

    def __init__(self):
        self.client = MongoClient(
            'mongodb://{}:{}@{}/{}'.format(MONGODB['user'],
                                           MONGODB['passwd'],
                                           MONGODB['host'],
                                           MONGODB['dbname']),
            connect=False
        )
        self.mongodb = self.client[MONGODB['dbname']]

    def __del__(self):
        self.client.close()

### 查询指定字段最新一条记录

In [ ]:
item_list = list(movie.find({'url': url}).sort({'_id': -1}).limit(1))

**注意:** limit返回的是一个迭代器,并不是数据结果

### 查询结果排序

In [ ]:
db.Account.find({}).sort("UserName")  --默认为升序
db.Account.find({}).sort("UserName",pymongo.ASCENDING)   --升序
db.Account.find({}).sort("UserName",pymongo.DESCENDING)  --降序

### 多列结果排序

In [ ]:
db.Account.find().sort([("UserName",pymongo.ASCENDING),("Email",pymongo.DESCENDING)])

### 关于mongodb你应该知道的

#### 索引

如果没有索引，MongoDB 需要为了找到一个匹配的文档而扫描整个 collection，代价非常高昂

#### 默认的_id索引

Mongodb 在 collection 创建时会默认建立一个基于`_id` 的唯一性索引作为 document 的 primary key，这个 index 无法被删除。

### 一些优化建议

1. 当无法使用索引或者无法有效利用`_id`索引时,也就是不得不进行全集合查询时,尽量减少全集合遍历查询次数.